In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICE'] = ''
import random

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from omegaconf import OmegaConf

%load_ext autoreload
%autoreload 2

from importlib.util import find_spec
if find_spec("text_recognizer") is None:
    import sys
    sys.path.append('..')

In [2]:
from hydra import compose, initialize
from omegaconf import OmegaConf
from hydra.utils import instantiate

In [3]:
path = "../training/conf/network/convnext.yaml"

In [4]:
with open(path, "rb") as f:
    cfg = OmegaConf.load(f)

In [5]:
cfg

{'_target_': 'text_recognizer.networks.convnext.ConvNext', 'dim': 16, 'dim_mults': [2, 4, 8], 'depths': [3, 3, 6], 'downsampling_factors': [[2, 2], [2, 2], [2, 2]], 'attn': {'_target_': 'text_recognizer.networks.convnext.TransformerBlock', 'attn': {'_target_': 'text_recognizer.networks.convnext.Attention', 'dim': 128, 'heads': 4, 'dim_head': 64, 'scale': 8}, 'ff': {'_target_': 'text_recognizer.networks.convnext.FeedForward', 'dim': 128, 'mult': 4}}}

In [10]:
cfg.dim_mults = [2, 4, 8, 8, 8]
cfg.depths = [3, 3, 3, 3, 6]
cfg.downsampling_factors = [[2, 2], [2, 2], [2, 2], [2, 1], [2, 1]]

In [11]:
net = instantiate(cfg)

In [12]:
from torchinfo import summary

In [13]:
summary(net, (2, 1, 576, 640), device="cpu", depth=4)

Layer (type:depth-idx)                             Output Shape              Param #
ConvNext                                           [2, 128, 18, 80]          5,969,376
├─Conv2d: 1-1                                      [2, 16, 576, 640]         800
├─ModuleList: 1                                    --                        --
│    └─ModuleList: 2                               --                        --
│    │    └─ConvNextBlock: 3-1                     [2, 16, 576, 640]         --
│    │    │    └─Conv2d: 4-1                       [2, 16, 576, 640]         800
│    │    │    └─Sequential: 4-2                   [2, 16, 576, 640]         9,280
│    │    │    └─Identity: 4-3                     [2, 16, 576, 640]         --
│    │    └─ModuleList: 3                          --                        --
│    │    │    └─ConvNextBlock: 4-4                [2, 16, 576, 640]         10,080
│    │    │    └─ConvNextBlock: 4-5                [2, 16, 576, 640]         10,080
│    │    │    